In [39]:
from settrade_v2 import Investor
from pprint import pprint
from settrade_v2 import Investor
import psycopg2
import pandas as pd
import time
from psycopg2 import sql
from datetime import datetime
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np

In [40]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [53]:
investor = Investor(
                app_id="SHX7ZfnIvozvpTRU",                                 
                app_secret="ALazfByX4KF34hZsIhw3uhPeGyfzklL0aYoj5SRJjQj9", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="FEEM-D")            

account_info = deri.get_account_info()

account_info       

{'creditLine': 10000000.0,
 'excessEquity': 10000000.0,
 'cashBalance': 10000000.0,
 'equity': 10000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 10000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [54]:
# กรณี Investor
market = investor.MarketData()


res = market.get_candlestick(
    symbol="BBL",
    interval="1d",
    limit=1,
    normalized=True,
)

print(res)

{'lastSequence': 439620, 'time': [1756746000], 'open': [152.5], 'high': [152.5], 'low': [152.0], 'close': [152.5], 'volume': [185100], 'value': [28207450.0]}


In [55]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,439620,1756746000,152.5,152.5,152.0,152.5,185100,28207450.0


In [63]:
session.set_keyspace('settrade_api')

In [66]:
session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol text,
        time timestamp,
        open_price float,
        high_price float,
        low_price float,
        close_price float,
        volume bigint,
        value float,
        PRIMARY KEY (symbol, time)
    ) WITH CLUSTERING ORDER BY (time ASC);
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

✅ Keyspace และ Table พร้อมใช้งาน!


In [67]:
# ✅ โหลดรายชื่อหุ้นทั้งหมด
try:
    symbols_df = pd.read_excel("C:/Foodbank/stock-pattern-detection-reverse-and-inverse-pattern-/stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'stocth_names.xlsx' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ ฟังก์ชันดึง candlestick จาก API
def get_candlestick_data(symbol):
    try:
        res = market.get_candlestick(  # ✅ ต้องกำหนด market ก่อนใช้
            symbol=symbol,
            interval="1d",
            limit=1000,
            normalized=True
        )
        return res
    except Exception as e:
        print(f"❌ ไม่สามารถดึงข้อมูลของ {symbol}: {e}")
        return None

# ✅ ฟังก์ชัน insert ข้อมูลลง Cassandra
def insert_financial_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        # ✅ แปลง timestamp ให้ถูกต้อง
        timestamp = datetime.fromtimestamp(res["time"][i])

        session.execute("""
            INSERT INTO candlestick_data (
                symbol, time, open_price, high_price, low_price,
                close_price, volume, value
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            timestamp,
            res["open"][i],
            res["high"][i],
            res["low"][i],
            res["close"][i],
            res["volume"][i],
            res["value"][i]
        ))

    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# ✅ ดึงและบันทึกข้อมูลทุกหุ้น
for symbol in symbols:
    print(f"\n📦 กำลังโหลดข้อมูล {symbol} ...")
    res = get_candlestick_data(symbol)
    insert_financial_data(symbol, res)
    time.sleep(2)  # 🕒 ลดโอกาสถูก block จาก API

print("\n🎉 เสร็จสิ้นการโหลดข้อมูลทั้งหมด!")


📦 กำลังโหลดข้อมูล 24CS ...
✅ เพิ่มข้อมูล 646 รายการของหุ้น 24CS สำเร็จ!

📦 กำลังโหลดข้อมูล 2S ...
✅ เพิ่มข้อมูล 721 รายการของหุ้น 2S สำเร็จ!

📦 กำลังโหลดข้อมูล 3BBIF ...
✅ เพิ่มข้อมูล 722 รายการของหุ้น 3BBIF สำเร็จ!

📦 กำลังโหลดข้อมูล A ...
✅ เพิ่มข้อมูล 711 รายการของหุ้น A สำเร็จ!

📦 กำลังโหลดข้อมูล A5 ...
✅ เพิ่มข้อมูล 722 รายการของหุ้น A5 สำเร็จ!

📦 กำลังโหลดข้อมูล AAI ...
✅ เพิ่มข้อมูล 628 รายการของหุ้น AAI สำเร็จ!

📦 กำลังโหลดข้อมูล AAV ...
✅ เพิ่มข้อมูล 722 รายการของหุ้น AAV สำเร็จ!

📦 กำลังโหลดข้อมูล ABM ...
✅ เพิ่มข้อมูล 720 รายการของหุ้น ABM สำเร็จ!

📦 กำลังโหลดข้อมูล ACAP ...
✅ เพิ่มข้อมูล 193 รายการของหุ้น ACAP สำเร็จ!

📦 กำลังโหลดข้อมูล ACC ...
✅ เพิ่มข้อมูล 721 รายการของหุ้น ACC สำเร็จ!

📦 กำลังโหลดข้อมูล ACE ...
✅ เพิ่มข้อมูล 722 รายการของหุ้น ACE สำเร็จ!

📦 กำลังโหลดข้อมูล ACG ...
✅ เพิ่มข้อมูล 717 รายการของหุ้น ACG สำเร็จ!

📦 กำลังโหลดข้อมูล ADB ...
✅ เพิ่มข้อมูล 722 รายการของหุ้น ADB สำเร็จ!

📦 กำลังโหลดข้อมูล ADD ...
✅ เพิ่มข้อมูล 721 รายการของหุ้น ADD สำเร็จ!

📦 กำล

In [68]:
name_stock = "A5"
rows = session.execute(f"SELECT * FROM candlestick_data WHERE symbol = '{name_stock}'")


In [69]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)
df.head()


,symbol,time,open_price,high_price,low_price,close_price,volume,value
0,A5,2022-06-13,2.62,2.68,2.60,2.66,0.0,14288655
1,A5,2022-06-14,2.62,2.66,2.58,2.60,0.0,7067511
2,A5,2022-06-15,2.58,2.62,2.58,2.62,0.0,9951951
3,A5,2022-06-16,2.38,2.66,2.36,2.60,0.0,19420919
4,A5,2022-06-17,2.28,2.40,2.18,2.40,0.0,10894701


In [70]:
import plotly.graph_objects as go

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    name=name_stock
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    name='Volume'
))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    title=f'📈 {name_stock} - Candlestick Chart with Volume',
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()


In [73]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- Step 1: Load Data ---
# ใช้ df ที่มีอยู่แล้วใน notebook

# --- Step 2: หา Swing Points ---
def detect_swings(df, window=5):
    swings = []
    for i in range(window, len(df)-window):
        if df['high_price'][i] == max(df['high_price'][i-window:i+window]):
            swings.append((i, df['high_price'][i], 'H'))
        elif df['low_price'][i] == min(df['low_price'][i-window:i+window]):
            swings.append((i, df['low_price'][i], 'L'))
    return swings

swings = detect_swings(df)

# --- Step 3: ตรวจจับ Harmonic Pattern ---
def is_gartley(xa, ab, bc, cd):
    return (0.618*xa <= ab <= 0.618*xa) and (0.382*ab <= bc <= 0.886*ab) and (0.786*xa <= cd <= 0.786*xa)

# (ต่อยอดได้อีกหลายแพทเทิร์น)

# --- Step 4: Visualization ---
fig = go.Figure(data=[go.Candlestick(x=df['time'],
                                     open=df['open_price'], high=df['high_price'],
                                     low=df['low_price'], close=df['close_price'])])
for i, price, typ in swings:
    fig.add_trace(go.Scatter(x=[df['time'][i]], y=[price],
                             mode="markers+text",
                             text=typ, textposition="top center"))

fig.show()
